In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.NRMS import NRMSModel
from configs.ManualConfig import hparams

In [2]:
hparams['name'] = 'nrms'
hparams['device'] = 'cuda:0'

In [3]:
vocab, loaders = prepare(hparams)

[2021-04-17 11:02:28,076] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:0', 'attrs': ['title'], 'k': 0, 'select': None, 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'name': 'nrms'}
[2021-04-17 11:02:28,077] INFO (root) preparing dataset...
[2021-04-17 11:02:31,922] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [4]:
nrmsModel = NRMSModel(hparams=hparams, vocab=vocab).to(hparams['device'])

In [55]:
nrmsModel(record)

torch.Size([5, 50, 256]) torch.Size([5, 5, 256]) torch.Size([5, 256])


tensor([[-1.5982, -1.6332, -1.6183, -1.6070, -1.5911],
        [-1.6151, -1.5789, -1.6114, -1.6313, -1.6112],
        [-1.6132, -1.6058, -1.6396, -1.5953, -1.5940],
        [-1.5935, -1.6218, -1.6280, -1.6071, -1.5973],
        [-1.6085, -1.6076, -1.6081, -1.6122, -1.6108]],
       grad_fn=<LogSoftmaxBackward>)

In [5]:
train(nrmsModel, hparams, loaders)

[2021-04-17 11:02:47,881] INFO (root) training...
epoch 1 , step 290 , loss: 1.5714: : 295it [00:10, 27.26it/s]
[2021-04-17 11:03:00,628] INFO (root) saved model of step 0, epoch 1 at data/model_params/nrms/demo_epoch1_step0_[hs=50,topk=0,attrs=title].model
epoch 2 , step 290 , loss: 1.4784: : 295it [00:09, 30.59it/s]
[2021-04-17 11:03:12,364] INFO (root) saved model of step 0, epoch 2 at data/model_params/nrms/demo_epoch2_step0_[hs=50,topk=0,attrs=title].model
epoch 3 , step 290 , loss: 1.4055: : 295it [00:09, 30.50it/s]
[2021-04-17 11:03:24,063] INFO (root) saved model of step 0, epoch 3 at data/model_params/nrms/demo_epoch3_step0_[hs=50,topk=0,attrs=title].model
epoch 4 , step 290 , loss: 1.3417: : 295it [00:09, 29.79it/s]
[2021-04-17 11:03:35,954] INFO (root) saved model of step 0, epoch 4 at data/model_params/nrms/demo_epoch4_step0_[hs=50,topk=0,attrs=title].model
epoch 5 , step 290 , loss: 1.2893: : 295it [00:09, 31.98it/s]
[2021-04-17 11:03:47,236] INFO (root) saved model of ste

NRMSModel(
  (encoder): MHA_Encoder(
    (embedding): Embedding(107392, 300, sparse=True)
    (MHA): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
    )
    (queryProject): Linear(in_features=300, out_features=256, bias=True)
    (keyProject): Linear(in_features=256, out_features=200, bias=True)
    (DropOut): Dropout(p=0.2, inplace=False)
  )
  (user_encoder): MHA_User_Encoder(
    (MHA): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
    )
    (keyProject): Linear(in_features=256, out_features=200, bias=True)
    (DropOut): Dropout(p=0.2, inplace=False)
  )
)